In [16]:
!pip install pillow transformers torch requests beautifulsoup4 google-search-results
!pip install openai
!pip install python-dotenv
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [29]:
import random
import textwrap
from PIL import Image, ImageDraw, ImageFont
from transformers import pipeline
from io import BytesIO
import requests
import os
import cv2

generator = pipeline('text-generation', model='gpt2')


def generate_caption(prompt="funny meme about"):

    generated_text = generator(prompt, max_length=len(prompt.split()) + 10, num_return_sequences=1)[0]['generated_text']
    return generated_text.strip()


def download_image(search_query):
    search_url = f"https://www.google.com/search?tbm=isch&q={search_query}"
    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    image_elements = soup.find_all('img')
    image_urls = [img['src'] for img in image_elements if 'src' in img.attrs]

    if not image_urls:
        raise Exception("No images found for the search query.")

    random_image_url = random.choice(image_urls)

    image_response = requests.get(random_image_url)
    img = Image.open(BytesIO(image_response.content))

    return img


def add_caption_to_image(img, caption, output_path):
    draw = ImageDraw.Draw(img)

    try:
        font = ImageFont.truetype("arial.ttf", 40)
    except IOError:
        font = ImageFont.load_default()

    margin = 10
    image_width = img.size[0]
    lines = textwrap.wrap(caption, width=40)

    y_text = img.size[1] - len(lines) * 45 - 20

    for line in lines:
        bbox = draw.textbbox((0, 0), line, font=font)
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]
        draw.text(((image_width - width) / 2, y_text), line, font=font, fill="white", stroke_fill="black", stroke_width=2)
        y_text += height

    img.save(output_path)


def extract_frame_from_video(video_path):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    if success:
        frame_path = "video_frame.jpg"
        cv2.imwrite(frame_path, image)
        vidcap.release()
        return frame_path
    else:
        raise Exception("Unable to extract a frame from the video.")


def request_file_path(file_type):
    file_path = input(f"Please provide the full path to your {file_type}: ").strip()
    if not os.path.exists(file_path):
        raise Exception(f"{file_type.capitalize()} path is invalid or does not exist.")
    return file_path


def generate_meme(search_query, output_path, prompt="funny meme about", use_own_image=False, image_path=None, video_path=None):
    if use_own_image:
        if image_path:
            img = Image.open(image_path)
        elif video_path:
            frame_path = extract_frame_from_video(video_path)
            img = Image.open(frame_path)
        else:
            raise Exception("No image or video provided.")
    else:
        img = download_image(search_query)

    caption = generate_caption(prompt)
    add_caption_to_image(img, caption, output_path)
    print(f"Meme generated and saved to {output_path}")

if __name__ == "__main__":
    use_own_media = input("Do you want to use your own image or video? (yes/no): ").strip().lower()

    if use_own_media == 'yes':
        media_type = input("Is it an image or a video? (image/video): ").strip().lower()

        if media_type == 'image':
            image_path = request_file_path('image')
            output_path = "generated_meme_from_image.jpg"
            prompt = input("Enter the prompt for the meme: ")
            generate_meme(None, output_path, prompt=prompt, use_own_image=True, image_path=image_path)

        elif media_type == 'video':
            video_path = request_file_path('video')
            output_path = "generated_meme_from_video.jpg"
            prompt = input("Enter the prompt for the meme: ")
            generate_meme(None, output_path, prompt=prompt, use_own_image=True, video_path=video_path)

        else:
            raise Exception("Invalid media type selected.")
    else:
        search_query = input("Enter the search query for the image: ")
        prompt = input("Enter the prompt for the meme: ")
        output_path = "generated_meme.jpg"
        generate_meme(search_query, output_path, prompt=prompt)


Do you want to use your own image or video? (yes/no): no
Enter the search query for the image: sunny leone
Enter the prompt for the meme: i love sex


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Meme generated and saved to generated_meme.jpg
